In [1]:
from collections import defaultdict
import pandas as pd
from tqdm import tqdm
import yaml
import random as rd
import numpy as np

In [2]:
def read_yaml(file_path):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

ds = read_yaml(file_path = "configs.yaml")

In [3]:
ds_config = "MovieLens100K"

In [4]:
train_data = pd.read_csv(f"datasets/{ds[ds_config]['ds_name']}/{ds[ds_config]['ds_name']}_train.txt", sep='\t', names=['uid', 'iid', 'rating'])
test_data = pd.read_csv(f"datasets/{ds[ds_config]['ds_name']}/{ds[ds_config]['ds_name']}_test.txt", sep='\t', names=['uid', 'iid', 'rating'])
long_tail_items = pd.read_csv(f"datasets/{ds[ds_config]['ds_name']}/groups/items/020/longtail_items.txt", names=['iid'])
short_head_items = pd.read_csv(f"datasets/{ds[ds_config]['ds_name']}/groups/items/020/shorthead_items.txt", names=['iid'])
active_users = pd.read_csv(f"datasets/{ds[ds_config]['ds_name']}/groups/users/005/active_ids.txt", names=['uid'])
inactive_users = pd.read_csv(f"datasets/{ds[ds_config]['ds_name']}/groups/users/005/inactive_ids.txt", names=['uid'])

In [5]:
users_ids = train_data['uid'].unique()
items_ids = train_data['iid'].unique()
num_users = len(train_data['uid'].unique())
num_items = len(train_data['iid'].unique())
active_users_list = list(active_users['uid'].unique())
inactive_users_list = list(inactive_users['uid'].unique())
longtail_items_list = list(long_tail_items['iid'].unique())
shorthead_items_list = list(short_head_items['iid'].unique())

In [6]:
pop_items = dict()

for eachline in train_data.iterrows():
    uid, iid = eachline[1][0], eachline[1][0]
    uid, iid = int(uid), int(iid)
    # a dictionary of popularity of items
    if iid in pop_items.keys():
      pop_items[iid] += 1
    else:
      pop_items[iid] = 1

In [7]:
def precisionk(actual, predicted):
  return 1.0 * len(set(actual) & set(predicted)) / len(predicted)

def recallk(actual, predicted):
  return 1.0 * len(set(actual) & set(predicted)) / len(actual)

def ndcgk(actual, predicted):
  idcg = 1.0
  dcg = 1.0 if predicted[0] in actual else 0.0
  for i, p in enumerate(predicted[1:]):
    if p in actual:
      dcg += 1.0 / np.log(i+2)
    idcg += 1.0 / np.log(i+2)
  return dcg / idcg

def novelty(predicted: list, pop: dict, u: int, k: int) -> float:
  self_information = 0
  for item in predicted:
    if item in pop.keys():
      item_popularity = pop[item] / u
      item_novelty_value = np.sum(-np.log2(item_popularity))
    else:
      item_novelty_value = 0
    self_information += item_novelty_value
  novelty_score = self_information / k
  return novelty_score

def catalog_coverage(predicted: list, catalog: list) -> float:
  predicted_flattened = [p for sublist in predicted for p in sublist]
  L_predictions = len(set(predicted_flattened))
  catalog_coverage = round(L_predictions / (len(catalog) * 1.0), 2)
  return catalog_coverage

In [8]:
# read test data
def read_ground_truth():
  ground_truth = defaultdict(set)
  for eachline in test_data.iterrows():
    uid, iid = eachline[1][0], eachline[1][1]
    uid, iid = int(uid), int(iid)
    ground_truth[uid].add(iid)
  return ground_truth

ground_truth = read_ground_truth()

In [9]:
def metric_per_group(group):
  # group: a list of users ids

  NDCG10 = list()
  Pre10 = list()
  Rec10 = list()
  Novelty10 = list()
  predicted = list()
  All_Predicted = list()

  long_items = 0
  short_items = 0
  all_items = 0

  for uid in tqdm(group):
    if uid in ground_truth.keys():
      # print(uid)
      for item in top_n_random[uid]:
        all_items += 1
        if item in longtail_items_list:
          long_items += 1
        if item in shorthead_items_list:
          short_items += 1
        predicted.append(item)

      copy_predicted = predicted[:] 
      All_Predicted.append(copy_predicted)
      NDCG = ndcgk(actual=ground_truth[uid], predicted=predicted)
      Pre = precisionk(actual=ground_truth[uid], predicted=predicted)
      Rec = recallk(actual=ground_truth[uid], predicted=predicted)
      Novelty = novelty(predicted=predicted, pop=pop_items, u=num_users, k=10)

      NDCG10.append(NDCG)
      Pre10.append(Pre)
      Rec10.append(Rec)
      Novelty10.append(Novelty)
    
      # cleaning the predicted list for a new user
      predicted.clear()

  catalog = catalog_coverage(predicted=All_Predicted, catalog=pop_items.keys())
  return round(np.mean(NDCG10), 5), round(np.mean(Pre10), 5), round(np.mean(Rec10), 5), round(np.mean(Novelty10), 5), catalog, short_items, long_items, all_items

In [10]:
def get_top_n_random(n=10):
  print("Random model is selected:")
  top_n = defaultdict(list)
  items = list(items_ids)
  for uid in users_ids:
    if uid not in top_n.keys():
      # user_id = list(rs.train_set.user_ids)[uid]
      for i in range(0, n):
        top_n[int(uid)].append(int(rd.choice(items)))
  return top_n

top_n_random = get_top_n_random()

Random model is selected:


In [11]:
result_file = open(f"{ds_config}_random.txt", "w")
for groups in [('active', active_users_list), ('inactive', inactive_users_list), ('all', users_ids)]:
    results = metric_per_group(group=groups[1])
    result_file.write(f"{groups[0]}: {results}\n")
result_file.close()

100%|██████████| 943/943 [00:01<00:00, 517.70it/s]
